In [62]:
from dotenv import dotenv_values
from pprint import pprint

from espn_api.football import League

from fantasy_sim.roster import ActiveRoster, HeapPlayer

import heapq

In [2]:
config = dotenv_values('.env')

In [3]:
league_id = 76723515
year = 2023

In [4]:
league = League(league_id=league_id, year=year, espn_s2=config['espn_s2'], swid=config['SWID'])

In [5]:
league

League(76723515, 2023)

In [6]:
league.load_roster_week(1)

In [7]:
league.teams

[Team(CeeDee Lambchops),
 Team(Let 'Em Cook),
 Team(Baby Chark doo-doo),
 Team(Gibbs you hell),
 Team(Burrowing to Last Place),
 Team(Out of Favre),
 Team(Mixon Dixons),
 Team(Di**er?! I hardly know 'er!)]

In [8]:
my_team = league.teams[1]

In [9]:
my_team.roster

[Player(Christian McCaffrey),
 Player(Garrett Wilson),
 Player(Patrick Mahomes),
 Player(Najee Harris),
 Player(Amari Cooper),
 Player(DeAndre Hopkins),
 Player(Rachaad White),
 Player(Kyle Pitts),
 Player(Tyler Lockett),
 Player(Dalvin Cook),
 Player(George Pickens),
 Player(Justin Tucker),
 Player(Saints D/ST),
 Player(Jakobi Meyers),
 Player(Raheem Mostert),
 Player(Dalton Schultz),
 Player(Aaron Rodgers),
 Player(Dolphins D/ST),
 Player(Darnell Mooney)]

In [10]:
box_scores = league.box_scores(1)

In [11]:
box_scores[0]

Box Score(Team(Out of Favre) at Team(Burrowing to Last Place))

In [12]:
for player in box_scores[0].home_lineup:
    print(player.position)

WR
RB
WR
QB
RB
WR
WR
TE
RB
WR
RB
WR
WR
D/ST
D/ST
K
QB
TE
WR
WR


In [13]:
test_roster = ActiveRoster()

In [14]:
test_lineup = box_scores[0].home_lineup

In [44]:
def build_player_heap(players):
    player_heap = [HeapPlayer(p) for p in players]
    heapq.heapify(player_heap)

    return player_heap

In [45]:
def is_roster_filled(roster):
    return all(
        all(slot) if isinstance(slot, tuple) else slot is not None
        for slot in roster.values()
    )

In [69]:
FLEX_POS = ['RB', 'WR', 'TE', 'D/ST', 'K']

In [90]:
roster = {
    'QB': None,
    'RB': [None, None],
    'WR': [None, None],
    'TE': None,
    'FLEX': [None, None],
    'D/ST': None,
    'K': None
}

In [91]:
player_heap = build_player_heap(test_lineup)

In [92]:
while not is_roster_filled(roster) and player_heap:
    curr_player = heapq.heappop(player_heap)
    pos = curr_player.position
    slot = roster[pos]

    if slot is None:
        roster[pos] = curr_player
    elif pos == 'RB' and not all(slot):
        roster[pos][int(slot[0] is not None)] = curr_player
    elif pos == 'WR' and not all(slot):
        roster[pos][int(slot[0] is not None)] = curr_player
    elif pos in FLEX_POS and not all(roster['FLEX']):
        slot = roster['FLEX']
        slot[int(slot[0] is not None)] = curr_player
    pprint(roster)
    print('-' * 5)

{'D/ST': Player(Cowboys D/ST, points:35.0, projected:7.19),
 'FLEX': [None, None],
 'K': None,
 'QB': None,
 'RB': [None, None],
 'TE': None,
 'WR': [None, None]}
-----
{'D/ST': Player(Cowboys D/ST, points:35.0, projected:7.19),
 'FLEX': [None, None],
 'K': None,
 'QB': None,
 'RB': [Player(Aaron Jones, points:26.7, projected:15.53), None],
 'TE': None,
 'WR': [None, None]}
-----
{'D/ST': Player(Cowboys D/ST, points:35.0, projected:7.19),
 'FLEX': [None, None],
 'K': None,
 'QB': None,
 'RB': [Player(Aaron Jones, points:26.7, projected:15.53),
        Player(Bijan Robinson, points:20.3, projected:17.76)],
 'TE': None,
 'WR': [None, None]}
-----
{'D/ST': Player(Cowboys D/ST, points:35.0, projected:7.19),
 'FLEX': [None, None],
 'K': None,
 'QB': None,
 'RB': [Player(Aaron Jones, points:26.7, projected:15.53),
        Player(Bijan Robinson, points:20.3, projected:17.76)],
 'TE': None,
 'WR': [Player(Zay Jones, points:16.5, projected:8.74), None]}
-----
{'D/ST': Player(Cowboys D/ST, point

In [86]:
pprint(roster)

{'D/ST': Player(Cowboys D/ST, points:35.0, projected:7.19),
 'FLEX': [Player(David Montgomery, points:13.4, projected:12.28),
          Player(Khalil Herbert, points:11.4, projected:11.38)],
 'K': Player(Harrison Butker, points:8.0, projected:8.41),
 'QB': Player(Daniel Jones, points:4.46, projected:17.51),
 'RB': [Player(Aaron Jones, points:26.7, projected:15.53),
        Player(Bijan Robinson, points:20.3, projected:17.76)],
 'TE': Player(George Kittle, points:4.9, projected:10.48),
 'WR': [Player(Zay Jones, points:16.5, projected:8.74),
        Player(A.J. Brown, points:14.9, projected:14.56)]}
